In [ ]:
%matplotlib inline

import numpy as np
from scipy import stats, optimize
from matplotlib import pyplot as plt

from sklearn import gaussian_process
from sklearn.gaussian_process import kernels

import pickle
from urllib.request import urlopen

# Gaussian Processes: Part 2

## Exercise 9.1: Modeling CO₂ levels

The amount of carbon dioxide in the atmosphere has been measured continuously at the Mauna Loa observatory, Hawaii.
In this problem, you should use a Gaussian process to model the data from 1958 to 2003, and see how well that model can be used for predicting the data from 2004-2019.
They present their latest data at [their homepage](https://www.esrl.noaa.gov/gmd/ccgg/trends/), but for your convenience you can use the data in the format available [here](https://github.com/gpschool/labs/raw/2019/.resources/mauna_loa).
You can load the data with the following code snippet:

In [ ]:
# download data
data = pickle.load(
    urlopen("https://github.com/gpschool/labs/raw/2019/.resources/mauna_loa")
)

# extract observations and test data
x = data['X'].flatten()
y = data['Y'].flatten()
xtest = data['Xtest'].flatten()
ytest = data['Ytest'].flatten()

Here, `x` and `y` contain your training data.

Start exploring some simple kernels, and thereafter you may have a look at page 118-122 of the book [Gaussian processes for machine learning](http://www.gaussianprocess.org/gpml/chapters/RW5.pdf) for some inspiration on how to design a more bespoke kernel for this problem.

## Exercise 9.2: Learning hyperparameters


Until now, we have made GP regression using predefined hyperparameters, such as the lengthscale $\ell$ and noise variance $\sigma^2$.
In this exercise, we will estimate $\ell$ and $\sigma^2$ from the data by maximizing the marginal likelihood.
The logarithm of the marginal likelihood for a Gaussian process observed with Gaussian noise is

$$
\log p(\mathbf{y} \,|\, \mathbf{x}; \ell, \sigma_f^2, \sigma^2) = \log \mathcal{N}(\mathbf{y} \,|\, 0, \mathbf{K}_y) = -\frac{1}{2} \mathbf{y}^\mathsf{T} \mathbf{K}_y^{-1} \mathbf{y} - \frac{1}{2} \log{|\mathbf{K}_y|} - \frac{n}{2}\log{(2\pi)}
$$

where $\mathbf{K}_y = \kappa(\mathbf{x}, \mathbf{x}) + \sigma^2 \mathbf{I}$.

### (a)

Write a function that takes $\mathbf{x}$, $\mathbf{y}$, $\ell$, $\sigma_f^2$, and $\sigma^2$ as inputs and produces the logarithm of the marginal likelihood as output for the squared exponential covariance function.

### (b)

Consider the same data as before. Use $\sigma_f^2 = 1$ and $\sigma^2 = 0$ and compute the logarithm of the marginal likelihood for values of $\ell$ between 0.1 and 1 and plot it.
What seems to be the maximal value of the marginal likelihood on this interval?
Do GP regression based on this value of $\ell$.

## Exercise 9.3: Learning hyperparameters II

In this exercise we investigate a setting where the marginal likelihood has multiple local minima.

### (a)

Now, consider the following data

$$
\mathbf{x} = [−5, −3, 0, 0.1, 1, 4.9, 5]^\mathsf{T}, \qquad
\mathbf{y} = [0, −0.5, 1, 0.7, 0, 1, 0.7]^\mathsf{T}
$$

and compute the log marginal likelihood for both $\ell$ and $\sigma^2$.
Use a logarithmic 2D-grid for values of $\ell$ spanning from $10^{−1}$ to $10^2$ and for $\sigma^2$ spanning from $10^{−2}$ to $10^0$.
Visualize the marginal likelihood on that grid with a contour plot.

### (b)

Find the hyperparameters $\ell$ and $\sigma^2$ that correspond to the maximal marginal likelihood.
Perform GP regression on the data using these hyperparameters.

### (d)

Perform GP regression for the hyperparameters that correspond to other possible local optima of the marginal likelihood.
What differences do you see in your posterior?

## Exercise 9.4: Learning hyperparameters III

We repeat Exercise 9.3, but this time we estimate $\ell$ and $\sigma^2$ that maximize the marginal likelihood with the `fit()` function in scikit-learn automatically.
Consider the same data as in Exercise 9.3(a) and use, as before, the RBF kernel and measurement noise together.

### (a)

You still have to provide an initial value of the hyperparameters.
Try $\ell = 1$ and $\sigma^2 = 0.1$.
What hyperparameters do you get when optimizing?
Plot the corresponding mean and credibility regions.

### (b)

Try instead to initialize with $\ell = 10$ and $\sigma^2 = 1$.
What do you get now?

### (c)

Try to explain what happens by making a grid over different hyperparameter values, and inspect the marginal likelihood for each point in that grid.
The `GaussianProcessRegressor` class has a member function `log_marginal_likelihood()` which you may use.
(Do not forget to turn off the hyperparameter optimization!)